In [1]:
import pandas as pd
import numpy as np
import collections
from copy import copy
from collections import defaultdict

import sys
sys.path.append('/dados01/workspace/ufmg.f01dcc/utils')
import google_drive

In [2]:
# google_drive.shareFile('1_Gm2b6zbrQ3yeWsWWH38ifcWZDFivtkc', 'owner')

In [3]:
# Creating folders through Google Drive API

# files_folder_id = google_drive.createFolder('Arquivos API')
# f03_folder = '1ojxfUGkh6KqSc09BWaOwQoWn1YYXhPcE'
# forms_folder = google_drive.createFolder('Formulários (JSON)', f03_folder)

In [4]:
forms_folder = '17xtCDOyF3Wk_q3X5TquR-O2nXr9G0hgv'

In [5]:
minimum_size = 20
item_name = 'mascara'

In [6]:
clusters = pd.read_csv("../dados/output/cluster_prices_statistics_year.csv.zip", sep=';')
clusters = clusters[clusters.cluster.str.contains(item_name)]
items = pd.read_csv("../dados/output/items_clusters_train.csv.zip", sep=';')
items = items[(items.outlier != 1) & (items.cluster.str.contains(item_name))]

In [7]:
clusters.head()

,cluster,dsc_unidade_medida,ano,mean,count,max,min,median,std,var,quantile_1,quantile_3
302460,mascara_0,caixa,2016,9.500,1,9.50,9.50,9.500,NaN,NaN,9.5000,9.5000
302461,mascara_0,caixa,2017,7.165,2,7.45,6.88,7.165,0.403051,0.16245,7.0225,7.3075
302462,mascara_0,caixa,2018,8.850,1,8.85,8.85,8.850,NaN,NaN,8.8500,8.8500
302463,mascara_0,p1,2015,4.175,2,4.60,3.75,4.175,0.601041,0.36125,3.9625,4.3875
302464,mascara_0,pa,2017,5.800,1,5.80,5.80,5.800,NaN,NaN,5.8000,5.8000


In [8]:
valid_clusters = clusters.groupby(['cluster'])['count'].sum()
valid_clusters = valid_clusters[valid_clusters > minimum_size]
print('Grupos distintos:', len(clusters))
print('Grupos distintos com mais de 20 itens:', len(valid_clusters))
print('Total de itens:', valid_clusters.sum())

Grupos distintos: 892
Grupos distintos com mais de 20 itens: 82
Total de itens: 8007


In [9]:
un_medidas = defaultdict(int)

clusters_un_medida = clusters.groupby(['cluster','dsc_unidade_medida'])['count'].sum()
clusters_un_medida = clusters_un_medida[clusters_un_medida > minimum_size]

for cluster_info in clusters_un_medida.index.values:
    un_medidas[cluster_info[1]] += 1

print('Unidades de medida distintas: ', len(un_medidas))

for key,value in un_medidas.items():
    print(f'{key}: {round(value/sum(un_medidas.values()),2)}')

Unidades de medida distintas:  5
unidade: 0.78
caixa: 0.16
caixa 50 unidades: 0.02
pacote: 0.02
peca: 0.01


In [10]:
valid_clusters = clusters.groupby(['cluster', 'dsc_unidade_medida'])['count'].sum()
valid_clusters = valid_clusters.sort_values(ascending=False)

In [11]:
json_info = {}

json_info['item_name'] = 'Máscara'
json_info['distinct_groups_count'] = int(len(clusters))
json_info['valid_groups_count'] = int(len(valid_clusters))
json_info['total_items'] = int(valid_clusters.sum())
json_info['distinct_units'] = int(len(un_medidas))

json_info['units'] = {}
for key,value in un_medidas.items():
    json_info['units'][key] = round(value/sum(un_medidas.values()),2)
    
json_info['clusters'] = []

In [12]:
for cluster_info in valid_clusters.index.values[:5]:
    print('-'*40)
    cluster_dict = {}
    
    cluster_name = cluster_info[0]
    cluster_un_medida = cluster_info[1]
    cluster_dict['name'] = cluster_name
    cluster_dict['unit'] = cluster_un_medida
    
    print(f'## {cluster_name} ##')
    print(f'Unidade de medida: {cluster_un_medida}')
    
    cluster_items = items[(items.cluster == cluster_name) & (items.dsc_unidade_medida == cluster_un_medida)]
    cluster_descriptions = cluster_items.groupby('description').size().sort_values(ascending=False)
    total_descriptions = cluster_descriptions.values.sum()
    
    print('Tamanho únicas: ', len(cluster_items))
    print('Descrições únicas: ', len(cluster_items['description'].unique()))
    print('Descrições mais frequentes:', list(cluster_descriptions.index.values)[:5])
    
    cluster_dict['size'] = int(len(cluster_items))
    cluster_dict['distinct_descriptions'] = int(len(cluster_items['description'].unique()))
    cluster_dict['total_descriptions'] = int(cluster_descriptions.values.sum())
    cluster_dict['descriptions'] = {}
    
    for desc in cluster_descriptions.index.values[:5]:
        cluster_dict['descriptions'][desc] = round(cluster_descriptions[desc]/total_descriptions,2)

    cluster_tokens = defaultdict(int)
    for item in cluster_descriptions.index.values:
        for token in item.split(' '):
            cluster_tokens[token] += 1
    cluster_tokens = sorted(cluster_tokens.items(), key=lambda x: x[1], reverse=True)
    
    print('Tokens mais frequentes:', list(cluster_tokens)[:20])
    
    cluster_dict['tokens'] = dict(cluster_tokens[:20])
    json_info['clusters'].append(cluster_dict)

----------------------------------------
## mascara_26 ##
Unidade de medida: caixa
Tamanho únicas:  1304
Descrições únicas:  351
Descrições mais frequentes: ['mascara descartavel', 'mascara cirurgico descartavel', 'mascara cirurgico', 'mascara descartavel com elastico', 'mascara descartavel c elastico']
Tokens mais frequentes: [('mascara', 352), ('descartavel', 209), ('elastico', 193), ('50', 182), ('c', 169), ('com', 152), ('cirurgico', 119), ('caixa', 80), ('cx', 75), ('unidade', 72), ('100', 55), ('unid', 51), ('branco', 35), ('desc', 33), ('un', 23), ('und', 23), ('descartar', 22), ('tnt', 14), ('odontologico', 11), ('cor', 10)]
----------------------------------------
## mascara_26 ##
Unidade de medida: unidade
Tamanho únicas:  925
Descrições únicas:  312
Descrições mais frequentes: ['mascara descartavel', 'mascara a95', 'mascara', 'mascara n 95', 'mascara descartavel azul']
Tokens mais frequentes: [('mascara', 312), ('descartavel', 96), ('com', 43), ('elastico', 41), ('c', 38), (

In [13]:
json_info

{'item_name': 'Máscara',
 'distinct_groups_count': 892,
 'valid_groups_count': 349,
 'total_items': 8007,
 'distinct_units': 5,
 'units': {'unidade': 0.78,
  'caixa': 0.16,
  'caixa 50 unidades': 0.02,
  'pacote': 0.02,
  'peca': 0.01},
 'clusters': [{'name': 'mascara_26',
   'unit': 'caixa',
   'size': 1304,
   'distinct_descriptions': 351,
   'total_descriptions': 1304,
   'descriptions': {'mascara descartavel': 0.11,
    'mascara cirurgico descartavel': 0.07,
    'mascara cirurgico': 0.03,
    'mascara descartavel com elastico': 0.02,
    'mascara descartavel c elastico': 0.02},
   'tokens': {'mascara': 352,
    'descartavel': 209,
    'elastico': 193,
    '50': 182,
    'c': 169,
    'com': 152,
    'cirurgico': 119,
    'caixa': 80,
    'cx': 75,
    'unidade': 72,
    '100': 55,
    'unid': 51,
    'branco': 35,
    'desc': 33,
    'un': 23,
    'und': 23,
    'descartar': 22,
    'tnt': 14,
    'odontologico': 11,
    'cor': 10}},
  {'name': 'mascara_26',
   'unit': 'unidade',
 

In [14]:
google_drive.writeJsonFile('mascaras', json_info, forms_folder)

Permission Id: 11918420385910730359
File ID: 1JvEa7d0xRO1GCCcIPMJGD9T2H30Uki1B


'1JvEa7d0xRO1GCCcIPMJGD9T2H30Uki1B'

In [ ]:
google_drive.readFiles()